# Notas sobre HQ-SAM

## Treinamento

### Dataset

O dataset é composto por imagens de diferentes dimensões que são redimensionados para 1024x1024, ou para a resolução passada pela flag `--input_size` ao rodar o script de treino.

As anotações do dataset são as máscaras de segmentação dos objetos. 
Não tem anotado especificamente os tokens de entrada (bounding box, pontos ou noise_mask).

O que acontece é que a partir das máscaras ground truth, são geradas em tempo de treino tokens de entrada aleatórios.

In [ ]:
## train.py ln 407 - ln 4016 ##
input_keys = ['box','point','noise_mask']
labels_box = misc.masks_to_boxes(labels[:,0,:,:])
try:
    labels_points = misc.masks_sample_points(labels[:,0,:,:])
except:
    # less than 10 points
    input_keys = ['box','noise_mask']
    labels_256 = F.interpolate(labels, size=(256, 256), mode='bilinear')
    labels_noisemask = misc.masks_noise(labels_256)

Após gerar os tokens, um tipo aleatório é escolhido para cada imagem:

In [ ]:
## train.py ln 417 - ln 434 ##
batched_input = []
for b_i in range(len(imgs)):
    dict_input = dict()
    input_image = torch.as_tensor(imgs[b_i].astype(dtype=np.uint8), device=sam.device).permute(2, 0, 1).contiguous()
    dict_input['image'] = input_image 
    input_type = random.choice(input_keys)
    if input_type == 'box':
        dict_input['boxes'] = labels_box[b_i:b_i+1]
    elif input_type == 'point':
        point_coords = labels_points[b_i:b_i+1]
        dict_input['point_coords'] = point_coords
        dict_input['point_labels'] = torch.ones(point_coords.shape[1], device=point_coords.device)[None,:]
    elif input_type == 'noise_mask':
        dict_input['mask_inputs'] = labels_noisemask[b_i:b_i+1]
    else:
        raise NotImplementedError
    dict_input['original_size'] = imgs[b_i].shape[:2]
    batched_input.append(dict_input)

Pelo menos durante o treino, a capacidade do SAM de gerar múltiplas mascáras de saída é desligada.

In [ ]:
## train.py ln 436 - ln 437 ##
with torch.no_grad():
    batched_output, interm_embeddings = sam(batched_input, multimask_output=False)